In [1]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from gensim.models import FastText 
from tqdm import tqdm

#Impotamos los csvs

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1slJiG-M_oawvkzaqWzZkSmVxxZor6mKQ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_features.csv')

id='1TmL2AY_yymiV8zXVvmbqXsIZoexR9sLs'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_features.csv')

id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

id='1GsTM9oLtIV8-Da_fDOFWsQYMpgQ8GOYJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

In [4]:
train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Limpiar texto

In [5]:
import re
import nltk
def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    porter_stemmer=nltk.PorterStemmer()
    cleaned_str = ''
    for word in words:
      cleaned_str += str(porter_stemmer.stem(word))
    return words

In [6]:
train['text'] = train['text'].apply(lambda x: " ".join((Tokenizer(x))))       
test['text'] = test['text'].apply(lambda x: " ".join((Tokenizer(x))))       

In [7]:
train['text'] = train['text'].fillna(' ')
test['text'] = test['text'].fillna(' ')
train['keyword'] = train['keyword'].fillna(' ')
test['keyword'] = test['keyword'].fillna(' ')

#Encoding con Universal Sentence Encoder



In [8]:
import tensorflow_hub as hub
import tensorflow as tf
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
train_tokens = [tf.reshape(embed([line]), [-1]).numpy() for line in train.text]
test_tokens = [tf.reshape(embed([line]), [-1]).numpy() for line in test.text]

#Matriz solamente con el texto codificado

In [13]:
matrix_use_text_train = pd.DataFrame(np.matrix(train_tokens))
matrix_use_text_test = pd.DataFrame(np.matrix(test_tokens))

In [14]:
matrix_use_text_train.to_csv('matriz_train_use_solo_text.csv', index=False)
files.download('matriz_train_use_solo_text.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
matrix_use_text_test.to_csv('matriz_test_use_solo_text.csv', index=False)
files.download('matriz_test_use_solo_text.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Matriz con texto y features numericos

In [19]:
numeric_features_test = test_features[['cant_stop_words', 'prom_long_palabra', 'cant_puntuacion', 'cant_apariciones_keyword', 'cant_numeros', 'cant_mayusculas', 'cant_vocales']]
numeric_features_train = train_features[['cant_stop_words', 'prom_long_palabra', 'cant_puntuacion', 'cant_apariciones_keyword', 'cant_numeros', 'cant_mayusculas', 'cant_vocales']]

In [21]:
use_features_numericos_train = pd.concat([matrix_use_text_train, numeric_features_train], axis=1, sort=False)
use_features_numericos_test = pd.concat([matrix_use_text_test, numeric_features_test], axis=1, sort=False)

In [22]:
use_features_numericos_train.to_csv('use_features_numericos_train.csv', index=False)
use_features_numericos_test.to_csv('use_features_numericos_test.csv', index=False)

In [23]:
files.download('use_features_numericos_train.csv')
files.download('use_features_numericos_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>